In [7]:
import pandas as pd
import statsmodels.formula.api as sm
from sklearn.linear_model import RandomizedLasso
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
dfNumeric = df.select_dtypes(include=['int64'])
dfCategoric = df.select_dtypes(include=['object'])
df.shape

(1460, 81)

In [9]:
df_Dummy = pd.get_dummies(df)

In [10]:
columns = df_Dummy.columns
columns = ','.join(columns).replace('.','').replace('1st','first').replace('2nd','second').replace(' ','').replace('3S','threeS').replace('(','_').replace(')','').replace('&','_and_')
df_Dummy.columns = columns.split(',')
columns = df_Dummy.columns.difference(["SalePrice"])
columns = " + ".join(columns)
my_formula = 'SalePrice ~ ' + columns                
results = sm.ols(formula = my_formula, data = df_Dummy).fit()
print(results.summary())
# Not a good method as interaction between predicotrs cannot be judged just by p values.

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     54.70
Date:                Sat, 17 Sep 2016   Prob (F-statistic):               0.00
Time:                        15:05:29   Log-Likelihood:                -12733.
No. Observations:                1121   AIC:                         2.595e+04
Df Residuals:                     879   BIC:                         2.717e+04
Df Model:                         241                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept              -1.80

In [11]:
## Using randomisedLasso - Stability to compute the important features
rlasso = RandomizedLasso(alpha=0.025)
df_Lasso = df_Dummy.dropna()
rlasso.fit(df_Lasso[df_Lasso.columns.difference(["SalePrice"])] , df_Lasso['SalePrice'])
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), 
                 df_Lasso.columns.difference(["SalePrice"])), reverse=True)

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: Depre

Features sorted by their score:
[(1.0, 'OverallQual'), (1.0, 'Neighborhood_StoneBr'), (1.0, 'Neighborhood_NridgHt'), (1.0, 'Neighborhood_NoRidge'), (1.0, 'KitchenQual_Ex'), (0.995, 'Neighborhood_Crawfor'), (0.995, 'LotConfig_CulDSac'), (0.99, 'TotalBsmtSF'), (0.99, 'Neighborhood_Edwards'), (0.99, 'MasVnrArea'), (0.99, 'Functional_Typ'), (0.99, 'ExterQual_Ex'), (0.99, 'BsmtFinSF1'), (0.985, 'OverallCond'), (0.985, 'GarageCars'), (0.985, 'Fireplaces'), (0.985, 'BsmtExposure_Gd'), (0.98, 'YearRemodAdd'), (0.98, 'KitchenAbvGr'), (0.975, 'MSZoning_C_all'), (0.975, 'LotArea'), (0.975, 'Exteriorfirst_BrkFace'), (0.965, 'YearBuilt'), (0.965, 'BsmtExposure_No'), (0.96, 'WoodDeckSF'), (0.96, 'BsmtQual_Ex'), (0.955, 'ScreenPorch'), (0.955, 'Neighborhood_NWAmes'), (0.95, 'Neighborhood_NAmes'), (0.95, 'FullBath'), (0.945, 'RoofMatl_WdShngl'), (0.945, 'Neighborhood_Mitchel'), (0.935, 'LotConfig_FR2'), (0.93, 'SaleCondition_Abnorml'), (0.925, 'Condition1_Norm'), (0.92, 'MSZoning_FV'), (0.92, 'LandSlo

In [56]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
clf = LassoCV()
clf.fit(df_Lasso[df_Lasso.columns.difference(["SalePrice"])] , df_Lasso['SalePrice'])

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [45]:
# use feature importance for feature selection
from math import log
from numpy import sort
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error

# split data into X and y
X = df_Lasso[df_Lasso.columns.difference(["SalePrice"])]
Y = df_Lasso['SalePrice']
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
# fit model on all training data
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
RMSE = mean_squared_error(map(log,y_test), map(log,y_pred))**0.5
print("RMSE: %.2f%%" % (RMSE))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
mini = RMSE
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = ExtraTreesClassifier()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    RMSE = mean_squared_error(map(log,y_test), map(log,y_pred))**0.5
    if mini > RMSE: 
        mini = RMSE
    print("Thresh=%.3f, n=%d, RMSE: %.2f%%" % (thresh, select_X_train.shape[1], RMSE))
print mini

RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.35%
Thresh=0.000, n=289, RMSE: 0.31%
Thresh=0.000, n=289, RMSE: 0.35%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.30%
Thresh=0.000, n=289, RMSE: 0.31%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.35%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.31%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.34%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.30%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.31%
Thresh=0.000, n=289, RMSE: 0.34%
Thresh=0.000, n=289, RMSE: 0.31%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.33%
Thresh=0.000, n=289, RMSE: 0.34%
Thresh=0.000, n=289, RMSE: 0.32%
Thresh=0.000, n=289, RMSE: 0.34

In [44]:
model = LassoCV()
model.fit(X_train, y_train)
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
RMSE = mean_squared_error(map(log,y_test), map(log,y_pred))**0.5
print("RMSE: %.5f%%" % (RMSE))
# Fit model using each importance as a threshold
thresholds = sort(model.coef_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = LassoCV()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    RMSE = mean_squared_error(map(log,y_test), map(log,y_pred))**0.5
    print("Thresh=%.5f, n=%d, RMSE: %.5f%%" % (thresh, select_X_train.shape[1], RMSE))
#Choosing Thresh=10.07243 as it gives the least RMSE

RMSE: 0.18466%
Thresh=-14.42288, n=289, RMSE: 0.18466%
Thresh=-2.97316, n=289, RMSE: 0.18466%
Thresh=-1.90797, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, n=289, RMSE: 0.18466%
Thresh=-0.00000, 

In [46]:
-
###  2 ways stacking and averaging
### 


In [47]:
y_pred_LinearRegression 

array([ 220792.9492515 ,  232041.0603931 ,  134744.49928552,
        207113.53232751,  166438.68585079,   97797.12367392,
        187219.61757882,  214726.38098794,  139949.37126922,
        273943.25035033,  222688.4545574 ,  108703.00249308,
        173085.12054538,  227073.18854364,  107119.51069622,
        227870.20170972,  168280.84318291,  151388.0189862 ,
        175755.86074577,  145036.47058209,  257232.43097144,
        219062.5014009 ,  302622.97627837,  146021.90428621,
        160003.88917071,  240992.03949034,  210955.35515293,
        104181.78682845,  183977.31898992,  167901.4345027 ,
         79824.90470405,   66566.85945186,  131798.21381683,
        173610.11104779,  171183.23759588,  152390.67093347,
        204471.42900946,  227005.93159165,  305085.32397665,
        202117.23716567,   55798.05459792,  238867.01108553,
        135613.83017072,  181570.11014812,  120617.10537211,
        144443.20945391,  241409.37782765,  187153.87488695,
        176521.57060278,

In [51]:
print 'RandomForest: ' , mean_squared_error(map(log,y_test), map(log,y_pred_RandomForest))**0.5
print 'LASSO: ' , mean_squared_error(map(log,y_test), map(log,y_pred_Lasso))**0.5
print 'LassoCV: ' , mean_squared_error(map(log,y_test), map(log,y_pred_LassoCV))**0.5
print 'LinearRegression: ' , mean_squared_error(map(log,y_test), map(log,y_pred_LinearRegression))**0.5

RandomForest:  0.182423375777
LASSO:  0.181418551254
LassoCV:  0.179260148174
LinearRegression:  0.181418637691


In [55]:
combined = []
for i, e in enumerate(y_pred_LinearRegression):
    predic = ( y_pred_RandomForest[i] + y_pred_Lasso[i] + y_pred_LassoCV[i] + e ) / 4
    combined.append(predic)

import numpy as np
combinedArr = np.asarray(combined)
print 'Averaged: ' , mean_squared_error(map(log,y_test), map(log,combinedArr))**0.5

Averaged:  0.168474975438


In [123]:
import pandas as pd
import statsmodels.formula.api as sm
from math import log
from numpy import sort
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
# Models
from sklearn.linear_model import RandomizedLasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV


def modelSelector(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                        test_size=0.33,
                                                        random_state=7)
    model = LassoCV()
    model.fit(X_train, y_train)
    # make predictions for test data and evaluate
    y_pred = model.predict(X_test)
    RMSE = mean_squared_error(map(log, y_test), map(log, y_pred))**0.5
    # Fit model using each importance as a threshold
    selectedModel = SelectFromModel(model, threshold=0.0, prefit=True)
    minRMSE = RMSE
    thresholds = sort(model.coef_)
    for thresh in thresholds:
        # select features using threshold
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_X_train = selection.transform(X_train)
        # train model
        selection_model = LassoCV()
        selection_model.fit(select_X_train, y_train)
        # eval model
        select_X_test = selection.transform(X_test)
        y_pred = selection_model.predict(select_X_test)
        try:
            RMSE = mean_squared_error(y_test, y_pred)**0.5
            if RMSE <= minRMSE:
                selectedModel = selection
                minRMSE = RMSE
        except ValueError:
            print("Invalid number for log")
    return selectedModel


def modelCreator(X, Y):
    # These are the models used for initial modelling
    models = [LassoCV(max_iter=1500), Lasso(), RandomForestRegressor(),
              LinearRegression()]
    fittedModels = []
    for m in models:
        m.fit(X,Y)
        fittedModels.append(m)
    return fittedModels


def combineModels(fittedModels, X, y):
    df_y = pd.DataFrame()
    for i, m in enumerate(fittedModels):
        print m
        df_y['model'+str(i)] = pd.Series(list(m.predict(X)))
    finalModel = RandomForestRegressor()
    finalModel.fit(df_y, y)
    return finalModel


def applyModels(models, X):
    df_y = pd.DataFrame()
    for i, m in enumerate(models[0]):
        df_y['model'+str(i)] = pd.Series(list(m.predict(X)))
    return models[1][0].predict(df_y)


# The categorical values have been converted to numerical dummy variables
df_Dummy = pd.get_dummies(pd.read_csv('train.csv').drop('Id', 1)).dropna()
df_test = pd.get_dummies(pd.read_csv('test.csv'))


df_test_id = df_test.Id.copy()
df_test = df_test.drop('Id', 1).fillna(0)
# The number of sets the data needs to split. One for each level
kFoldSplit = 3
# split data into X and y
X = df_Dummy[df_Dummy.columns.difference(["SalePrice"])]
Y = df_Dummy['SalePrice']

col_to_add = np.setdiff1d(X.columns, df_test.columns)

# add these columns to test, setting them equal to zero
for c in col_to_add:
    df_test[c] = 0

firstLevelModels = []
# Models used for regression
X_train, X, y_train, y = train_test_split(X, Y,
                                          test_size=(2)/kFoldSplit,
                                          random_state=7)

#selectedModel = modelSelector(X_train, y_train)
firstLevelModels = modelCreator(X_train, y_train)

df_y = pd.DataFrame()
for i, m in enumerate(firstLevelModels):
    df_y['model'+str(i)] = pd.Series(list(m.predict(X_train)))
finalModel = RandomForestRegressor()
finalModel.fit(df_y, y_train)
modelsF = [firstLevelModels, [finalModel]]

Y = applyModels(modelsF, df_test)
df_final = pd.DataFrame()
df_final['SalePrice'] = pd.Series(Y)
df_final.index = df_test_id
df_final.to_csv('Output1.csv')


In [131]:
modelsF[0][1].predict(selectedModel.transform(df_test))

array([-19339817.28760519, -16726811.41633492, -15662171.80818807, ...,
       -20841615.95380045,  -1853474.29890784, -18094867.56342993])

In [100]:
print df_test.shape
print X_train.shape

(1459, 289)
(1121, 288)


In [135]:
def applyModels(models, X):
    df_y = pd.DataFrame()
    for i, m in enumerate(models[0]):
        df_y['model'+str(i)] = m.predict(X)
    return df_y.mean(axis=1)
Y = applyModels(modelsF, selectedModel.transform(df_test))

In [136]:
Y

0       9.186176e+14
1       1.078247e+15
2       1.222527e+15
3       1.218928e+15
4       1.044967e+15
5       1.248914e+15
6       1.013587e+15
7       1.169910e+15
8       1.048225e+15
9       9.183629e+14
10      1.087236e+15
11      1.163495e+15
12      1.196878e+15
13      1.152061e+15
14      9.043848e+14
15      1.481658e+15
16      1.213857e+15
17      1.372722e+15
18      1.289281e+15
19      1.873254e+15
20      1.467595e+15
21      1.133666e+15
22      1.060183e+15
23      1.012570e+15
24      1.027842e+15
25      1.274947e+15
26      1.524257e+15
27      1.219959e+15
28      1.035351e+15
29      1.107555e+15
            ...     
1429    8.387224e+14
1430    1.314472e+15
1431    6.201511e+14
1432    9.553479e+14
1433    8.351758e+14
1434    1.288996e+15
1435    1.204578e+15
1436    1.143194e+15
1437    1.153289e+15
1438    1.109501e+15
1439    1.074008e+15
1440    1.139350e+15
1441    1.077379e+15
1442    1.245395e+15
1443    1.339776e+15
1444    6.264899e+14
1445    1.116